In [1]:
import datetime
import math
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [2]:
tweets = pd.read_csv("./data/tweets_all.csv", encoding="latin1")

In [4]:
min_lat = tweets.LATITUDE.min()
min_long = tweets.LONGITUDE.min()

In [5]:
print(tweets.LATITUDE.max())
print(tweets.LATITUDE.min())
print(tweets.LONGITUDE.max())
print(tweets.LONGITUDE.min())

51.69515609741211
51.28390121459961
0.3412590026855469
-0.5209429860115051


In [6]:
lat_range = tweets.LATITUDE.max() - tweets.LATITUDE.min()
long_range = tweets.LONGITUDE.max() - tweets.LONGITUDE.min()

lat_bin_size = lat_range/9
long_bin_size= long_range/9

In [7]:
print(lat_range)
print(long_range)

0.4112548828125
0.862201988697052


In [8]:
print(lat_bin_size)
print(long_bin_size)

0.045694986979166664
0.09580022096633911


In [9]:
tweets["Space"] = tweets.LONGITUDE.apply(lambda long: (((long - min_long)//long_bin_size)+1))
tweets["Row"] = tweets.LATITUDE.apply(lambda lat: (9-((lat - min_lat)//lat_bin_size)+1))

In [10]:
tweets["Space"] = tweets["Space"].apply(lambda l: int(l))
tweets["Row"] = tweets["Row"].apply(lambda l: int(l))

In [11]:
tweets.head()

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET,Space,Row
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames; London,NaN,0,3,8
1,550226823314178048,NaN,-0.149466,51.492560,-0.149466,51.492561,31/12/2014 09:47:52,465989904,alice_foster_95,So. Many. Accents #alicesadventureswithmegabus,London; England,alicesadventureswithmegabus,0,4,6
2,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,New Addington; London,NaN,0,6,9
3,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Stratford; London,NaN,0,6,5
4,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Romford; London,NaN,0,8,4


Remove non-plaintext elements from tweets

In [14]:
tweets["One"] = 1

In [15]:
tweets.head()

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET,Space,Row,parsed_tweets,One
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames; London,NaN,0,3,8,Not yet got tics for Liverpool waiting till Fr...,1
1,550226823314178048,NaN,-0.149466,51.492560,-0.149466,51.492561,31/12/2014 09:47:52,465989904,alice_foster_95,So. Many. Accents #alicesadventureswithmegabus,London; England,alicesadventureswithmegabus,0,4,6,So Many Accents,1
2,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,New Addington; London,NaN,0,6,9,If it doesn t make you happy by December st le...,1
3,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Stratford; London,NaN,0,6,5,thanks and are you open tomorrow at all,1
4,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Romford; London,NaN,0,8,4,but he is,1


In [22]:
# Want to remove tweets with
# * Row = 1, Space = 8
# * Row = 3, Space = 8
# Only contain one tweet
tweets = tweets[~((tweets["Row"] == 1) & (tweets["Space"] == 8 ))]
tweets = tweets[~((tweets["Row"] == 3) & (tweets["Space"] == 10 ))]

In [20]:
tweets_by_lat_long = tweets.groupby(["Row", "Space", "One"]).id.agg(["count"])

In [21]:
tweets_by_lat_long.to_csv("./data/tweets_by_lat_long.csv")